### Non-masked

In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

In [2]:
Non_masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Non_masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Non_masked_edges_node_names_explainer_subgraphs.node1.tolist(), Non_masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Non_masked_edges_node_names_explainer_subgraphs.shape[0]



Non_masked_results_for_Heatmaps = pd.concat([genes, Non_masked_edges_node_names_explainer_subgraphs], axis=1)
Non_masked_results_for_Heatmaps


2022-11-18 06:35:07,366	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1,2,3,4,5,6,...,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400
0,[8140.1],r1622,phe_L_e,0.010364,0.973281,0.008578,0.981670,0.010527,0.010533,0.007768,...,0.011289,0.975524,0.967636,0.010913,0.009823,0.011705,0.011981,0.981386,0.035208,0.981847
1,[9057.1],r1898,ile_L_e,0.013913,0.395449,0.014437,0.441639,0.013735,0.013762,0.014078,...,0.013836,0.331057,0.302477,0.013736,0.014439,0.013623,0.013565,0.462284,0.027120,0.436932
2,"[6520.1, 23428.1]",LEUPHELAT2tc,leu_L_e,0.013917,0.711719,0.015004,0.539293,0.013928,0.014004,0.014572,...,0.013941,0.842872,0.695234,0.013993,0.014785,0.013914,0.013647,0.461872,0.029577,0.542636
3,"[6520.1, 23428.1]",LEUPHELAT2tc,phe_L_e,0.014826,0.978887,0.017719,0.979795,0.014306,0.014595,0.017169,...,0.014876,0.974521,0.968630,0.014606,0.016835,0.013488,0.013140,0.980907,0.147013,0.979622
4,"[6520.1, 23428.1]",LEUPHELAT2tc,phe_L_c,0.014486,0.738554,0.016355,0.251163,0.014551,0.014249,0.015445,...,0.014363,0.817997,0.642190,0.014253,0.015046,0.013952,0.013634,0.165406,0.167457,0.248580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,[8140.1],r1647,leu_L_e,0.013863,0.745213,0.014783,0.610934,0.013953,0.013934,0.014376,...,0.013730,0.826704,0.787393,0.013822,0.014362,0.013825,0.013502,0.498086,0.028798,0.569893
381,[9057.1],r1943,phe_L_c,0.013388,0.074007,0.012805,0.080938,0.013328,0.013275,0.013183,...,0.013447,0.074184,0.078544,0.013381,0.013130,0.013337,0.013248,0.080995,0.040436,0.076916
382,[9057.1],r1943,arg_L_e,0.014078,0.088870,0.014560,0.089456,0.013996,0.014103,0.014285,...,0.014029,0.085810,0.086664,0.013979,0.014573,0.013875,0.013798,0.086236,0.024030,0.090156
383,[9057.1],r1943,phe_L_e,0.011851,0.981878,0.011315,0.982174,0.011960,0.012033,0.011949,...,0.012319,0.982039,0.980334,0.012165,0.012225,0.012054,0.012376,0.982354,0.039923,0.982113


In [3]:
Non_masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Non_masked_results_for_Heatmaps.csv")

# Masked

In [4]:
Masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Masked_edges_node_names_explainer_subgraphs.node1.tolist(), Masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Masked_edges_node_names_explainer_subgraphs.shape[0]



Masked_results_for_Heatmaps = pd.concat([genes, Masked_edges_node_names_explainer_subgraphs], axis=1)
Masked_results_for_Heatmaps

2022-11-18 06:35:12,181	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1,2,3,4,5,6,...,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400
0,[8140.1],r1547,gly_c,0.013986,0.014022,0.335903,0.412890,0.014093,0.013764,0.013170,...,0.034599,0.014138,0.013899,0.022248,0.356503,0.177758,0.014200,0.014063,0.021424,0.021564
1,[213.1],ala_L_c,r1092,0.013826,0.013794,0.030831,0.973607,0.013727,0.013712,0.013157,...,0.975910,0.013781,0.013717,0.016004,0.827804,0.961536,0.013706,0.013848,0.014690,0.010657
2,[12.1],ala_L_c,r1093,0.013798,0.013726,0.063971,0.974602,0.013657,0.013795,0.013312,...,0.975162,0.013722,0.013780,0.015805,0.891694,0.961938,0.013751,0.013800,0.014694,0.010524
3,[5265.1],ala_L_c,r1094,0.013698,0.013856,0.025545,0.973813,0.013630,0.013801,0.013166,...,0.976243,0.013800,0.013842,0.015812,0.816665,0.961414,0.013788,0.013919,0.014692,0.010656
4,[338.1],ala_L_c,r1095,0.013740,0.013901,0.111594,0.976186,0.013829,0.013779,0.013087,...,0.974380,0.013662,0.013775,0.015766,0.927184,0.962095,0.013818,0.013711,0.014456,0.010618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,[],r1335,leu_L_c,0.013770,0.013863,0.969803,0.970330,0.013878,0.013840,0.013590,...,0.974572,0.013849,0.013812,0.013307,0.971673,0.974499,0.013719,0.013896,0.013466,0.013458
381,[],r1335,ile_L_c,0.013676,0.013835,0.970655,0.970719,0.013793,0.013792,0.013669,...,0.974879,0.013748,0.013644,0.013562,0.972352,0.975261,0.013805,0.014141,0.013396,0.018188
382,[],r1335,glu_L_c,0.013855,0.014011,0.079587,0.178400,0.013742,0.013737,0.013612,...,0.257090,0.014030,0.014003,0.014297,0.193390,0.407854,0.013815,0.013984,0.014297,0.013474
383,[8140.1],r1647,leu_L_c,0.013821,0.013872,0.384130,0.061179,0.013834,0.013727,0.013873,...,0.011899,0.013738,0.013720,0.013989,0.877522,0.889332,0.013830,0.013687,0.014183,0.013935


In [5]:
Masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Masked_results_for_Heatmaps.csv")